In [1]:
import qiskit
qiskit_version = int(qiskit.__version__.split(".")[0])
if qiskit_version == 1:
    from qiskit_aer import AerProvider
    env = "cloud"
else:
    from qiskit import Aer
    env = "local"

import numpy as np
import math
from scipy.sparse import coo_matrix
import networkx as nx
import random
import matplotlib.pyplot as plt
plt.style.use('default')

from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit.primitives import BackendSampler
from qiskit import QuantumRegister
from qiskit.circuit import QuantumCircuit


import json
import pandas as pd
from scipy.sparse import csr_array, csr_matrix
from sklearn import linear_model


from hydra import initialize, compose
from os.path import dirname, abspath
import sys
parent_dir = dirname(dirname(abspath(' ')))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
import src.encode.encode_permutation as encode_p
import src.encode.encode_combination as encode_c
#from src.bitflip_strategy.local_search_general import *
from src.bitflip_strategy.Local_Search_QUBO import local_search

from src.optimizer.Quantum_Optimizer_FGSR_b_hardware import *
#from src.optimizer.Quantum_Optimizer_FGSR_b import *
from src.optimizer.utils import *

from src.optimizer.hardware_utils import *

from maxcut_utils import *

import warnings
warnings.simplefilter('ignore')

from qiskit_ibm_runtime.fake_provider import FakeTorino, FakeSherbrooke
from qiskit.visualization import plot_gate_map, plot_error_map

In [2]:
# Load config file
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name='config_shimada.yaml')

In [3]:
# Set backend
if env == "cloud":
    service = QiskitRuntimeService(
        channel=cfg.hardware.channel,
        instance=cfg.hardware.instance,
        token=cfg.hardware.token
    )
    backend = service.get_backend(cfg.hardware.backend)
else:
    backend = Aer.get_backend(cfg.simulator.backend)

In [4]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_circuit_layout

In [5]:
num_qubits = 120
start = 14
gate_type = "cz"
chain_nodes, sep_nodes, sep_nodes_coupling = get_initial_layout(backend, num_qubits, start, gate_type)

In [6]:
print(len(chain_nodes))
print(chain_nodes)
print(len(sep_nodes))
print(sep_nodes)
print(sep_nodes_coupling)

112
[14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 37, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 53, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 75, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 91, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 113, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 129]
8
[92, 110, 132, 130, 93, 54, 18, 131]
[(112, 84), (113, 108), (114, 98), (115, 106), (116, 70), (117, 52), (118, 2), (119, 102)]


In [11]:
reps = 2
qc = create_ansatz(reps, chain_nodes, sep_nodes, sep_nodes_coupling)
qc.measure_all()

In [12]:
qc_t = transpile(qc, backend=backend, initial_layout=chain_nodes+sep_nodes)
print(qc_t.num_nonlocal_gates(), qc_t.num_nonlocal_gates()==(num_qubits-1)*reps)

238 True


In [14]:
layout_data = {}
num_qubits = 120
backend_name_list = ["ibm_torino", "ibm_sherbrooke", "ibm_osaka", "ibm_nazca", "ibm_brisbane", "ibm_kyoto", "ibm_cusco"]
start_list = [14, 13, 13, 13, 13, 13, 13]
gate_type_list = ["cz", "ecr", "ecr", "ecr", "ecr", "ecr", "ecr"]

In [17]:
for backend_name, start, gate_type in zip(backend_name_list, start_list, gate_type_list):
    print(backend_name, start, gate_type)
    backend = service.get_backend(backend_name)
    chain_nodes, sep_nodes, sep_nodes_coupling = get_initial_layout(backend, num_qubits, start, gate_type)
    qc = create_ansatz(reps, chain_nodes, sep_nodes, sep_nodes_coupling)
    qc.measure_all()
    qc_t = transpile(qc, backend=backend, initial_layout=chain_nodes+sep_nodes)
    if qc_t.num_nonlocal_gates()==(num_qubits-1)*reps:
        layout_data[backend_name] = [chain_nodes, sep_nodes, sep_nodes_coupling]
    else:
        print("error")

ibm_torino 14 cz
ibm_sherbrooke 13 ecr
ibm_osaka 13 ecr
ibm_nazca 13 ecr
ibm_brisbane 13 ecr
ibm_kyoto 13 ecr
ibm_cusco 13 ecr


In [22]:
import dill

with open('layout_data_120q.pickle', mode='wb') as f:
    dill.dump(layout_data, f)

In [21]:
layout_data.keys()

dict_keys(['ibm_torino', 'ibm_sherbrooke', 'ibm_osaka', 'ibm_nazca', 'ibm_brisbane', 'ibm_kyoto', 'ibm_cusco'])

In [2]:
import dill

with open('layout_data_120q.pickle', mode='rb') as f:
    layout_data = dill.load(f)

In [4]:
layout_data["ibm_brisbane"][0]

[13,
 12,
 11,
 10,
 9,
 8,
 7,
 6,
 5,
 4,
 3,
 2,
 1,
 0,
 14,
 18,
 19,
 20,
 33,
 39,
 38,
 37,
 52,
 56,
 57,
 58,
 71,
 77,
 76,
 75,
 90,
 94,
 95,
 96,
 109,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 112,
 108,
 107,
 106,
 105,
 104,
 103,
 102,
 101,
 100,
 99,
 98,
 91,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 74,
 70,
 69,
 68,
 67,
 66,
 65,
 64,
 63,
 62,
 61,
 60,
 53,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 36,
 32,
 31,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21]

In [5]:
layout_data["ibm_brisbane"][1]

[17, 15, 113, 78, 92, 55, 111, 73, 110, 35, 59]